In [215]:
from langchain_core.messages import SystemMessage, HumanMessage, RemoveMessage, AIMessage, AnyMessage
from langgraph.graph import StateGraph, START, END
from typing import List, Dict, Any, TypedDict, Annotated
from pydantic import BaseModel
from langgraph.constants import Send
import json
from operator import add

In [218]:
import os
import requests
from urllib.parse import urlparse
import urllib.request as libreq
import feedparser

def create_path_if_not_exists(path: str):
    """
    Creates a directory path if it doesn't exist.

    Args:
        path (str): The directory path to create.
    """
    if not os.path.exists(path):
        os.makedirs(path)


def arxiv_pdf_link_extractor(links: List[Dict[str, str]]) -> str | None:
    """
    Extracts the PDF link from a list of links.
    Args:
        links (List[Dict[str, str]]): A list of dictionaries containing link information.
    Returns:
        str | None: The PDF link if found, otherwise None.
    """

    link = [link['href'] for link in links if link.get('title', '') == 'pdf']
    if len(link) > 0:
        return link[0]
    return None

def arxiv_search(search_query: str, start: int = 0, max_results: int = 5) -> List[Dict[str, Any]]:
    """
    Searches the arXiv API for papers matching the search query.

    Args:
        search_query (str): The query to search for.
        start (int): The starting index for results.
        max_results (int): The maximum number of results to return.

    Returns:
        List[Dict[str, Any]]: A list of dictionaries containing paper titles, summaries, publish dates and URLs.
    """
    api_call = 'http://export.arxiv.org/api/query?search_query=all:%s&start=%i&max_results=%i' % (
        search_query.replace(' ', '+'), start, max_results)
    try:
        response = libreq.urlopen(api_call)
        if response.status != 200:
            raise Exception(f"Error fetching data from arXiv API: {response.status}")
        results = feedparser.parse(response.read())
        docs = [{
            'title': entry.title, 
            'summary': entry.summary,
            'published': entry.published,
            'url': arxiv_pdf_link_extractor(entry.links)
            } for entry in results.entries]
        return [doc for doc in docs if doc['url'] is not None]
    except Exception as e:
        print(e)
        return []
    

def download_pdf(url: str, save_dir: str):
    """
    Downloads a PDF file from a URL to a specified path.

    Args:
        url (str): The URL of the PDF file.
        save_path (str): The path where the PDF file will be saved.
    Returns:
        str: The path where the PDF file was saved.
    """
    # Extract the filename from the URL
    filename = os.path.basename(urlparse(url).path) + '.pdf'
    save_path = os.path.join(save_dir, filename)

    # Ensure the directory exists
    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    
    try:
        # Download the file
        response = requests.get(url, stream=True)
        if response.status_code != 200:
            raise Exception(f"Failed to download file: {response.status_code}")
        with open(save_path, 'wb') as file:
            for chunk in response.iter_content(chunk_size=1024):
                file.write(chunk)
        return save_path
    except Exception as e:
        print(f"Error downloading PDF: {e}")
        return None

In [222]:
class arXivDocSpec(TypedDict):
    title: str
    summary: str
    published: str
    url: str

class PapersRetrieveGraphState(BaseModel):
    search_query: str
    docs_specs: List[arXivDocSpec] = []
    metadata_files: Annotated[List[str], add] = []

class PapersRetrieveGraph:
    def __init__(
            self,
            docs_path,
            docs_metadata_path,
            memory=None,
            max_results: int = 5,
    ):
        self.docs_path = docs_path
        self.docs_metadata_path = docs_metadata_path
        self.memory = memory
        self.max_results = max_results

        create_path_if_not_exists(self.docs_path)
        create_path_if_not_exists(self.docs_metadata_path)

        self.build_graph()

    def search_node(self, state: PapersRetrieveGraphState):
        """
        Searches for papers on arXiv based on the search query in the state.
        """
        docs_specs = arxiv_search(state.search_query, max_results=self.max_results, start=0)
        return {'docs_specs': docs_specs}
    
    def download_router(self, state: PapersRetrieveGraphState):
        """
        Distributes docs specifications into download nodes.
        """
        return [Send("doc_downloader", doc_spec) for doc_spec in state.docs_specs]
    
    def doc_download_node(self, doc_spec: arXivDocSpec):
        """
        Downloads a PDF file based on the document specification provided and saves its metadata.
        """
        result = download_pdf(doc_spec['url'], self.docs_path)
        if result is not None:
            doc_spec['path'] = result
            # Save doc_spec to metadata json file
            metadata_file = os.path.join(self.docs_metadata_path, f"{os.path.basename(result).split('.')[0]}.json")
            with open(metadata_file, 'w') as f:
                json.dump(doc_spec, f, indent=4)
            return {'metadata_files': [metadata_file]}
        return {'metadata_files': []}
    
    def build_graph(self):
        """
        Builds the state graph for retrieving papers from arXiv.
        """
        workflow = StateGraph(PapersRetrieveGraphState)
        workflow.add_node('search_docs', self.search_node)
        workflow.add_node('doc_downloader', self.doc_download_node)

        workflow.add_edge(START, 'search_docs')
        workflow.add_conditional_edges('search_docs', self.download_router, ['doc_downloader'])
        workflow.add_edge('doc_downloader', END)

        self.graph = workflow.compile(checkpointer=self.memory)

    def run(self, search_query: str, thread_id: str = None):
        """
        Runs the graph with the provided search query.

        Args:
            search_query (str): The query to search for papers on arXiv.
        """
        config = {'configurable': {'thread_id': thread_id}} if thread_id else None
        return self.graph.invoke({'search_query': search_query}, config=config)
        

In [224]:
downloader = PapersRetrieveGraph(
    docs_path='../docs',
    docs_metadata_path='../docs_metadata',
    max_results=3,
)

In [225]:
downloader.run(search_query='Chain of Thought')

Error downloading PDF: Failed to download file: 404


{'search_query': 'Chain of Thought',
 'docs_specs': [{'title': 'Contrastive Chain-of-Thought Prompting',
   'summary': 'Despite the success of chain of thought in enhancing language model\nreasoning, the underlying process remains less well understood. Although\nlogically sound reasoning appears inherently crucial for chain of thought,\nprior studies surprisingly reveal minimal impact when using invalid\ndemonstrations instead. Furthermore, the conventional chain of thought does not\ninform language models on what mistakes to avoid, which potentially leads to\nmore errors. Hence, inspired by how humans can learn from both positive and\nnegative examples, we propose contrastive chain of thought to enhance language\nmodel reasoning. Compared to the conventional chain of thought, our approach\nprovides both valid and invalid reasoning demonstrations, to guide the model to\nreason step-by-step while reducing reasoning mistakes. To improve\ngeneralization, we introduce an automatic method t

# Experiments

In [1]:
from langchain_community.document_loaders import ArxivLoader

In [21]:
query = 'Chain of Thought Prompting for Large Language Models'
max_docs = 4

loader = ArxivLoader(
    query=query,
    load_max_docs=max_docs,
    load_all_available_meta=False,
    sortBy="submittedDate",
    sortOrder="descending",
)

docs = loader.get_summaries_as_docs()

In [22]:
len(docs)

3

In [17]:
[doc.page_content for doc in docs]

['Despite the success of chain of thought in enhancing language model\nreasoning, the underlying process remains less well understood. Although\nlogically sound reasoning appears inherently crucial for chain of thought,\nprior studies surprisingly reveal minimal impact when using invalid\ndemonstrations instead. Furthermore, the conventional chain of thought does not\ninform language models on what mistakes to avoid, which potentially leads to\nmore errors. Hence, inspired by how humans can learn from both positive and\nnegative examples, we propose contrastive chain of thought to enhance language\nmodel reasoning. Compared to the conventional chain of thought, our approach\nprovides both valid and invalid reasoning demonstrations, to guide the model to\nreason step-by-step while reducing reasoning mistakes. To improve\ngeneralization, we introduce an automatic method to construct contrastive\ndemonstrations. Our experiments on reasoning benchmarks demonstrate that\ncontrastive chain o

In [4]:
doc = docs[0]

In [23]:
[doc.metadata for doc in docs]

[{'Entry ID': 'http://arxiv.org/abs/2201.11903v6',
  'Published': datetime.date(2023, 1, 10),
  'Title': 'Chain-of-Thought Prompting Elicits Reasoning in Large Language Models',
  'Authors': 'Jason Wei, Xuezhi Wang, Dale Schuurmans, Maarten Bosma, Brian Ichter, Fei Xia, Ed Chi, Quoc Le, Denny Zhou'},
 {'Entry ID': 'http://arxiv.org/abs/2506.02878v2',
  'Published': datetime.date(2025, 6, 6),
  'Title': 'CoT is Not True Reasoning, It Is Just a Tight Constraint to Imitate: A Theory Perspective',
  'Authors': 'Jintian Shao, Yiming Cheng'},
 {'Entry ID': 'http://arxiv.org/abs/2403.03544v1',
  'Published': datetime.date(2024, 3, 6),
  'Title': 'Prompt Mining for Language-based Human Mobility Forecasting',
  'Authors': 'Hao Xue, Tianye Tang, Ali Payani, Flora D. Salim'}]

In [16]:
[doc.metadata for doc in docs]

[{'Entry ID': 'http://arxiv.org/abs/2311.09277v1',
  'Published': datetime.date(2023, 11, 15),
  'Title': 'Contrastive Chain-of-Thought Prompting',
  'Authors': 'Yew Ken Chia, Guizhen Chen, Luu Anh Tuan, Soujanya Poria, Lidong Bing'},
 {'Entry ID': 'http://arxiv.org/abs/2305.16582v2',
  'Published': datetime.date(2024, 3, 23),
  'Title': 'Beyond Chain-of-Thought, Effective Graph-of-Thought Reasoning in Language Models',
  'Authors': 'Yao Yao, Zuchao Li, Hai Zhao'},
 {'Entry ID': 'http://arxiv.org/abs/2506.02878v2',
  'Published': datetime.date(2025, 6, 6),
  'Title': 'CoT is Not True Reasoning, It Is Just a Tight Constraint to Imitate: A Theory Perspective',
  'Authors': 'Jintian Shao, Yiming Cheng'}]

In [30]:
import arxiv, asyncio

In [26]:
arxiv.SortCriterion.__members__

mappingproxy({'Relevance': <SortCriterion.Relevance: 'relevance'>,
              'LastUpdatedDate': <SortCriterion.LastUpdatedDate: 'lastUpdatedDate'>,
              'SubmittedDate': <SortCriterion.SubmittedDate: 'submittedDate'>})

In [37]:
async def fetch_results(query, max_docs):
    client = arxiv.Client()
    search = arxiv.Search(
        query=query,
        max_results=max_docs,
        sort_by=arxiv.SortCriterion.Relevance
    )

    docs = []
    for result in client.results(search):
        print("Title:", result.title)
        print("Summary:", result.summary)
        print("Published:", result.published)
        print("PDF URL:", result.pdf_url)
        print()
        docs.append({
            "title": result.title,
            "summary": result.summary,
            "published": result.published,
            "pdf_url": result.pdf_url
        })

    return docs


In [38]:
import asyncio

async def main():
    # Define query and max_docs
    query = "Chain of Thought Prompting"
    max_docs = 4

    # Run the asynchronous function
    docs_spec = await fetch_results(query, max_docs)
    return docs_spec

# Call the async function
docs_spec = await main()

Title: Chain-of-Thought Prompting Elicits Reasoning in Large Language Models
Summary: We explore how generating a chain of thought -- a series of intermediate
reasoning steps -- significantly improves the ability of large language models
to perform complex reasoning. In particular, we show how such reasoning
abilities emerge naturally in sufficiently large language models via a simple
method called chain of thought prompting, where a few chain of thought
demonstrations are provided as exemplars in prompting. Experiments on three
large language models show that chain of thought prompting improves performance
on a range of arithmetic, commonsense, and symbolic reasoning tasks. The
empirical gains can be striking. For instance, prompting a 540B-parameter
language model with just eight chain of thought exemplars achieves state of the
art accuracy on the GSM8K benchmark of math word problems, surpassing even
finetuned GPT-3 with a verifier.
Published: 2022-01-28 02:33:07+00:00
PDF URL: http

In [ ]:
docs_spec

In [154]:
client = arxiv.Client()

search = arxiv.Search(
  query = query,
  max_results = max_docs,
  sort_by = arxiv.SortCriterion.Relevance
)

results = client.results(search)

In [ ]:
import urllib.request as libreq
import feedparser

# Search parameters
search_query = 'Chain of Thought' # search for electron in all fields
start = 0                     # retreive the first 5 results
max_results = 5

api_call = 'http://export.arxiv.org/api/query?search_query=all:%s&start=%i&max_results=%i' % (search_query.replace(' ', '+'),
                                                     start,
                                                     max_results)

with libreq.urlopen(api_call) as url:
    r = url.read()

feed = feedparser.parse(r)
feed

[{'title': entry.title, 'url': [link['href'] for link in entry.links if link.get('title','')=='pdf'][0]} for entry in feed.entries]

{'bozo': False,
 'entries': [{'id': 'http://arxiv.org/abs/2311.09277v1',
   'guidislink': True,
   'link': 'http://arxiv.org/abs/2311.09277v1',
   'updated': '2023-11-15T18:54:01Z',
   'updated_parsed': time.struct_time(tm_year=2023, tm_mon=11, tm_mday=15, tm_hour=18, tm_min=54, tm_sec=1, tm_wday=2, tm_yday=319, tm_isdst=0),
   'published': '2023-11-15T18:54:01Z',
   'published_parsed': time.struct_time(tm_year=2023, tm_mon=11, tm_mday=15, tm_hour=18, tm_min=54, tm_sec=1, tm_wday=2, tm_yday=319, tm_isdst=0),
   'title': 'Contrastive Chain-of-Thought Prompting',
   'title_detail': {'type': 'text/plain',
    'language': None,
    'base': '',
    'value': 'Contrastive Chain-of-Thought Prompting'},
   'summary': 'Despite the success of chain of thought in enhancing language model\nreasoning, the underlying process remains less well understood. Although\nlogically sound reasoning appears inherently crucial for chain of thought,\nprior studies surprisingly reveal minimal impact when using in

In [199]:
r = libreq.urlopen(api_call)
r

In [196]:
[{'title': entry.title, 'url': [link['href'] for link in entry.links if link.get('title','')=='pdf'][0]} for entry in feed.entries]

[{'title': 'Contrastive Chain-of-Thought Prompting',
  'url': 'http://arxiv.org/pdf/2311.09277v1'},
 {'title': 'Beyond Chain-of-Thought, Effective Graph-of-Thought Reasoning in\n  Language Models',
  'url': 'http://arxiv.org/pdf/2305.16582v2'},
 {'title': 'CoT is Not True Reasoning, It Is Just a Tight Constraint to Imitate: A\n  Theory Perspective',
  'url': 'http://arxiv.org/pdf/2506.02878v2'},
 {'title': 'Table as Thought: Exploring Structured Thoughts in LLM Reasoning',
  'url': 'http://arxiv.org/pdf/2501.02152v1'},
 {'title': 'Symbolic Chain-of-Thought Distillation: Small Models Can Also "Think"\n  Step-by-Step',
  'url': 'http://arxiv.org/pdf/2306.14050v2'}]

In [ ]:
doc = next(results)

In [94]:
doc.download_pdf(dirpath='../docs')

'../docs/2311.09277v1.Contrastive_Chain_of_Thought_Prompting.pdf'

In [157]:
doc.__annotations__

{'entry_id': 'str',
 'updated': 'datetime',
 'published': 'datetime',
 'title': 'str',
 'authors': 'List[Author]',
 'summary': 'str',
 'comment': 'Optional[str]',
 'journal_ref': 'Optional[str]',
 'doi': 'Optional[str]',
 'primary_category': 'str',
 'categories': 'List[str]',
 'links': 'List[Link]',
 'pdf_url': 'Optional[str]',
 '_raw': 'feedparser.FeedParserDict'}

In [166]:
import requests
import os
from urllib.parse import urlparse

def download_pdf(url: str, save_dir: str):
    """
    Downloads a PDF file from a URL to a specified path.

    Args:
        url (str): The URL of the PDF file.
        save_path (str): The path where the PDF file will be saved.
    """
    # Extract the filename from the URL
    filename = os.path.basename(urlparse(url).path) + '.pdf'
    save_path = os.path.join(save_dir, filename)

    # Ensure the directory exists
    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    
    # Download the file
    response = requests.get(url, stream=True)
    if response.status_code == 200:
        with open(save_path, 'wb') as file:
            for chunk in response.iter_content(chunk_size=1024):
                file.write(chunk)
    return f"PDF downloaded: {save_path}"

In [163]:
os.path.basename(urlparse(doc.pdf_url).path)

'2404.04538v1'

In [167]:
download_pdf(url=doc.pdf_url, save_dir='../docs')

'PDF downloaded: ../docs/2404.04538v1.pdf'

In [78]:
for r in results:
    print('Title:', r.title)
    #print(r.summary)
    print('Published:', r.published)
    print('url:',r.pdf_url)
    print()

Title: Chain-of-Thought Prompting Elicits Reasoning in Large Language Models
Published: 2022-01-28 02:33:07+00:00
url: http://arxiv.org/pdf/2201.11903v6

Title: Contrastive Chain-of-Thought Prompting
Published: 2023-11-15 18:54:01+00:00
url: http://arxiv.org/pdf/2311.09277v1

Title: GCoT: Chain-of-Thought Prompt Learning for Graphs
Published: 2025-02-12 03:33:06+00:00
url: http://arxiv.org/pdf/2502.08092v2

Title: Soft-Prompting with Graph-of-Thought for Multi-modal Representation Learning
Published: 2024-04-06 07:39:44+00:00
url: http://arxiv.org/pdf/2404.04538v1



In [86]:
import sys
sys.path.append("..")

In [88]:
r.download_pdf(dirpath='../docs')

ConnectionResetError: [Errno 104] Connection reset by peer

In [85]:
r.entry_id

'http://arxiv.org/abs/2404.04538v1'

In [89]:
paper = next(arxiv.Client().results(arxiv.Search(id_list=[r.entry_id])))

ConnectionError: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))

In [79]:
results[0].__annotations__

{'entry_id': 'str',
 'updated': 'datetime',
 'published': 'datetime',
 'title': 'str',
 'authors': 'List[Author]',
 'summary': 'str',
 'comment': 'Optional[str]',
 'journal_ref': 'Optional[str]',
 'doi': 'Optional[str]',
 'primary_category': 'str',
 'categories': 'List[str]',
 'links': 'List[Link]',
 'pdf_url': 'Optional[str]',
 '_raw': 'feedparser.FeedParserDict'}

In [68]:
import arxiv
import asyncio

async def fetch_results_for_queries(queries, max_docs):
    """
    Asynchronously fetch results from arxiv.Search for a list of queries.

    Args:
        queries (list): List of query strings.
        max_docs (int): Maximum number of documents to fetch per query.

    Returns:
        list: A list of dictionaries with 'query' and 'doc' keys.
    """
    client = arxiv.Client()
    results = []

    async def fetch_single_query(query):
        print(f"Fetching results for query: {query}")
        search = arxiv.Search(
            query=query,
            max_results=max_docs,
            sort_by=arxiv.SortCriterion.Relevance
        )
        # Run the blocking generator in a separate thread
        docs = await asyncio.to_thread(list, client.results(search))
        print(f"Finished fetching results for query: {query}")
        return {"query": query, "docs": docs}

    tasks = [fetch_single_query(query) for query in queries]
    results = await asyncio.gather(*tasks)
    return results

# Example usage
async def main(queries, max_docs):
    results = await fetch_results_for_queries(queries, max_docs)
    return results

# To run the async function
# results = asyncio.run(main())

In [69]:
queries = ["Chain of Thought Prompting", "Large Language Models", "AI in Healthcare"]
max_docs = 4

docs_spec = await main(queries=queries, max_docs=max_docs)
print(len(docs_spec))

Fetching results for query: Chain of Thought Prompting
Fetching results for query: Large Language Models
Fetching results for query: AI in Healthcare
Finished fetching results for query: Large Language Models
Finished fetching results for query: AI in Healthcare
Finished fetching results for query: Chain of Thought Prompting
3


In [70]:
[{'query': query['query'], 'docs': [doc.title for doc in query['docs']]} for query in docs_spec]

[{'query': 'Chain of Thought Prompting',
  'docs': ['Chain-of-Thought Prompting Elicits Reasoning in Large Language Models',
   'Contrastive Chain-of-Thought Prompting',
   'GCoT: Chain-of-Thought Prompt Learning for Graphs',
   'Soft-Prompting with Graph-of-Thought for Multi-modal Representation Learning']},
 {'query': 'Large Language Models',
  'docs': ['Lost in Translation: Large Language Models in Non-English Content Analysis',
   'Cedille: A large autoregressive French language model',
   'How Good are Commercial Large Language Models on African Languages?',
   'Goldfish: Monolingual Language Models for 350 Languages']},
 {'query': 'AI in Healthcare',
  'docs': ['SoK: Security and Privacy Risks of Healthcare AI',
   'Threefold model for AI Readiness: A Case Study with Finnish Healthcare SMEs',
   'Reliable and Resilient AI and IoT-based Personalised Healthcare Services: A Survey',
   'Risk of AI in Healthcare: A Comprehensive Literature Review and Study Framework']}]

In [71]:
docs = [{'query': result['query'], 'doc': doc} for result in docs_spec for doc in result['docs']]

In [75]:
doc.pdf_url

AttributeError: 'Document' object has no attribute 'pdf_url'

## Doc Loader

In [99]:
from langchain_community.document_loaders import PyPDFLoader

In [151]:
path = '../docs/2201.11903v6.Chain_of_Thought_Prompting_Elicits_Reasoning_in_Large_Language_Models.pdf'

In [130]:
path = '../docs/2311.09277v1.Contrastive_Chain_of_Thought_Prompting.pdf'

In [134]:
loader = PyPDFLoader(path, extraction_mode='plain')
pages = []
async for page in loader.alazy_load():
    pages.append(page)

In [135]:
content = pages[0].page_content

In [136]:
for line in content.split('\n'):
    print(line.strip())

Contrastive Chain-of-Thought Prompting
Yew Ken Chia∗ 1,
DeCLaRe Guizhen Chen∗ 1, 2
Luu Anh Tuan2 Soujanya Poria
DeCLaRe Lidong Bing† 1
1DAMO Academy, Alibaba Group, Singapore
DeCLaRe Singapore University of Technology and Design
2Nanyang Technological University, Singapore
{yewken_chia, sporia}@sutd.edu.sg
{guizhen001, anhtuan.luu}@ntu.edu.sg
{yewken.chia, guizhen.chen, l.bing}@alibaba-inc.com
Abstract
Despite the success of chain of thought in en-
hancing language model reasoning, the under-
lying process remains less well understood. Al-
though logically sound reasoning appears in-
herently crucial for chain of thought, prior stud-
ies surprisingly reveal minimal impact when
using invalid demonstrations instead. Further-
more, the conventional chain of thought does
not inform language models on what mistakes
to avoid, which potentially leads to more er-
rors. Hence, inspired by how humans can
learn from both positive and negative exam-
ples, we propose contrastive chain of thought
to

In [106]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [137]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100,
)

In [138]:
texts = text_splitter.split_text(content)

In [139]:
[len(t) for t in texts]

[497,
 461,
 489,
 470,
 498,
 477,
 275,
 232,
 232,
 261,
 457,
 394,
 455,
 497,
 463,
 472,
 499,
 361]

In [141]:
texts[-10]

'"raw_a": "37.5%",                                                                                                                                                                                                   \n"pred_b": "36.67%",'

In [140]:
for text in texts:
    for line in text.split('\n'):
        print(line)
    print('-' * 80)

Contrastive Chain-of-Thought Prompting
Yew Ken Chia∗ 1,
DeCLaRe Guizhen Chen∗ 1, 2
Luu Anh Tuan2 Soujanya Poria
DeCLaRe Lidong Bing† 1
1DAMO Academy, Alibaba Group, Singapore
DeCLaRe Singapore University of Technology and Design
2Nanyang Technological University, Singapore
{yewken_chia, sporia}@sutd.edu.sg
{guizhen001, anhtuan.luu}@ntu.edu.sg
{yewken.chia, guizhen.chen, l.bing}@alibaba-inc.com
Abstract
Despite the success of chain of thought in en-
hancing language model reasoning, the under-
--------------------------------------------------------------------------------
Despite the success of chain of thought in en-
hancing language model reasoning, the under-
lying process remains less well understood. Al-
though logically sound reasoning appears in-
herently crucial for chain of thought, prior stud-
ies surprisingly reveal minimal impact when
using invalid demonstrations instead. Further-
more, the conventional chain of thought does
not inform language models on what mistakes
to av

In [152]:
loader = PyPDFLoader(path, mode='single', extraction_mode='plain')
content = loader.load()[0].page_content

In [153]:
texts = text_splitter.split_text(content)
print(len(texts))

342
